# AutoCprrector Version 2


|<h1>Preparation des données</h1> |
| :------------------: |
| process_data : Divise le corpus en phrases et les phrases en mots.
count_words : Compte la fréquence de chaque mot.
get_words_with_frequency_above_or_equal N: Identifie les mots fréquents.
prepare_data : Remplace les mots rares par un jeton spécifié et retourne des nouvelles phrases tokenisées et traitées|

In [21]:
import re
from collections import defaultdict
from typing import List, Tuple
import math

def process_data(data: str) -> Tuple[List[str], List[List[str]]]:
    """
    Tokenize les données en phrases et en mots.

    Paramètres :
        data (str) : Les données d'entrée sous forme de chaîne de caractères.

    Retourne :
        Tuple[List[str], List[List[str]]] : Un tuple contenant la liste des phrases et la liste des phrases tokenisées.
    """
    def tokenizer_phrase(phrase: str) -> List[str]:
        phrase = phrase.lower()
        phrase = re.sub(r"[^\w\s]", "", phrase)
        tokens = phrase.split()
        return tokens

    phrases = data.split('\n')
    phrases = [phrase.strip() for phrase in phrases if phrase.strip()]
    phrases_tokenisees = [tokenizer_phrase(p) for p in phrases]
    
    return phrases, phrases_tokenisees

def count_words(phrases_tokenisees: List[List[str]]) -> dict:
    """
    Compte le nombre d'apparitions de chaque mot dans les phrases tokenisées.

    Paramètres :
        sentences_tokenises (list[list[str]]) : Liste de listes de chaînes de caractères.

    Retourne :
        word_counts (dict[str, int]) : Dictionnaire qui fait correspondre chaque mot (str) à sa fréquence (int).
    """
    word_counts = {}
    for phrase in phrases_tokenisees:
        for token in phrase:
            if token not in word_counts.keys():
                word_counts[token] = 1
            else:
                word_counts[token] += 1

    return word_counts

def get_words_with_frequency_above_or_equal(phrases_tokenisees: List[List[str]], 
                               threshold_frequency: int) -> List[str]:
    """
    Trouve les mots qui apparaissent N fois ou plus.

    Paramètres :
        tokenized_sentences (list[list[str]]) : Liste de listes de phrases.
        threshold_frequency (int) : Nombre minimum d'occurrences pour qu'un mot fasse partie du vocabulaire restreint.

    Retourne :
        closed_vocab (list[str]) : Liste des mots qui apparaissent N fois ou plus.
    """
    closed_vocab = []
    word_counts = count_words(phrases_tokenisees)
    for word, count in word_counts.items():
        if count >= threshold_frequency:
            closed_vocab.append(word)

    return closed_vocab

def prepare_data(phrases_tokenisees: List[List[str]], 
                         closed_vocab: List[str], unknown_token: str="<unk>"
                         ) -> List[List[str]]:
    """
    Replace les mots qui ne sont pas dans le vocabulaire donné par le jeton inconnu.

    Paramètres:
        phrases_tokenisees: Liste de listes de chaînes de caractères
        closed_vocab: Liste de chaînes de caractères que nous utiliserons
        unknown_token: Une chaîne de caractères représentant les mots inconnus (hors vocabulaire)
    
    Retourne:
        replaced_phrases_tokenisees: Liste de listes de chaînes de caractères, avec les mots qui ne sont pas dans le vocabulaire remplacés
    """
    closed_vocab = set(closed_vocab)  # Convertir en ensemble
    replaced_phrases_tokenisees = []
    for phrase in phrases_tokenisees:
        replaced_phrase = []
        for token in phrase:
            if token in closed_vocab:
                replaced_phrase.append(token)
            else:
                replaced_phrase.append(unknown_token)
        replaced_phrases_tokenisees.append(tuple(replaced_phrase))  # Convertir en tuple
        
    return replaced_phrases_tokenisees




|<h1>Entrainement des modèles n-grames </h1> |
| :------------------: |
|
<h3>count_n_grams() : </h3> Cette fonction compte tous les n-grammes dans les données fournies. Elle prend une liste de tuples de mots, un entier n représentant la taille de l'n-gramme, ainsi que des tokens de début et de fin de phrase optionnels. Elle retourne un dictionnaire qui mappe chaque n-gramme à sa fréquence.<h3>train(): </h3> Cette fonction entraîne un modèle de n-gramme avec un lissage add-k à partir d'un fichier d'entrée. Elle lit d'abord le contenu du fichier, puis prétraite les données en tokenisant les phrases, en remplaçant les mots hors vocabulaire, et en comptant les n-grammes. Ensuite, elle calcule les probabilités de transition avec le lissage add-k et les stocke dans un dictionnaire, qui est ensuite retourn..|

In [22]:
def count_n_grams(data: List[Tuple[str]], n: int, start_token: str='<s>', end_token: str= '</s>') -> dict:
    """
    Compte tous les n-grammes dans les données fournies.
    
    Paramètres :
        data (list[tuple[str]]) : Liste de tuples de mots.
        n (int) : nombre de mots dans une séquence (n-gramme).
        start_token (str) : une chaîne de caractères indiquant le début de la phrase (par défaut '<s>').
        end_token (str) : une chaîne de caractères indiquant le fin de la phrase (par défaut '</s>').
    
    Retourne :
        n_grams (dict) : Un dictionnaire qui mappe un tuple de n mots à sa fréquence.
    """
    n_grams = {}

    for phrase in data:
        phrase = (start_token,) * (n - 1) + phrase + (end_token,)
        for i in range(len(phrase) - n + 1): 
            n_gram = phrase[i:i+n]
            n_gram = tuple(phrase[i:i+n])
            if n_gram in n_grams:
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1
    
    return n_grams

def train(infile: str, ngram_size: int, k: float) -> dict:
    """
    Entraîne un modèle de n-gramme avec lissage add-k à partir d'un fichier d'entrée.
    """
    with open(infile, "r", encoding="utf-8") as file:
        data = file.read()

    _, phrases_tokenisees = process_data(data)
    vocab = get_words_with_frequency_above_or_equal(phrases_tokenisees, 1)
    replaced_phrases = prepare_data(phrases_tokenisees, vocab)

    ngrams = count_n_grams(replaced_phrases, ngram_size)
    vocab_size = len(set(word for phrase in phrases_tokenisees for word in phrase))

    # Calcul des probabilités avec lissage add-k et normalisation
    ngram_probs = {}
    for ngram, count in ngrams.items():
        context = " ".join(ngram[:-1])
        word = ngram[-1]
        context_count = sum(value for key, value in ngrams.items() if key[:-1] == ngram[:-1])
        prob = (count + k) / (context_count + k * vocab_size)
        
        if context in ngram_probs:
            ngram_probs[context][word] = prob 
        else:
            ngram_probs[context] = {word: prob }

   

    return ngram_probs

|<h2>Prédiction d'une phrase </h2> |
| :------------------: |
|
<h3>predict_ngram() : </h3> Cette fonction est utilisée pour calculer la probabilité d'une phrase donnée en utilisant un modèle de n-gramme. 
Elle commence par normaliser la phrase d'entrée en la divisant en tokens et en les convertissant en minuscules. Ensuite, elle itère à travers les n-grammes de la phrase (en prenant en compte la taille de l'n-gramme spécifiée). Pour chaque n-gramme, elle récupère le contexte et le mot actuel, et vérifie s'ils existent dans le modèle de n-gramme. Si c'est le cas, elle ajoute le logarithme de la probabilité du n-gramme au total des probabilités logarithmiques. Enfin, elle retourne la somme totale des probabilités logarithmiques calculées.|

In [23]:
def predict_ngram(sentence: str, ngram_model: dict, ngram_size: int) -> float:
    """
    Calcule la probabilité d'une phrase donnée en utilisant un modèle de n-gramme.

    Paramètres :
        sentence (str) : La phrase dont on veut calculer la probabilité.
        ngram_model (dict) : Le modèle de n-gramme entraîné, contenant les probabilités des n-grammes.
        ngram_size (int) : La taille des n-grammes à utiliser dans le modèle.

    Retourne :
        float : La probabilité logarithmique de la phrase selon le modèle de n-gramme.
    """
    # Normalisation de la phrase
    tokens = sentence.lower().split()
    n = len(tokens)

    total_log_prob = 0.0
    for i in range(ngram_size - 1, n):
        context = " ".join(tokens[i - ngram_size + 1:i])
        word = tokens[i]
        if context in ngram_model and word in ngram_model[context]:
            total_log_prob += math.log(ngram_model[context][word])
        else:
            pass

    return total_log_prob

|<h2>Evaluer le modèle à travers la perplexité </h2> |
| :------------------: |
|
<h3>normalize_log_prob() : </h3>Cette fonction prend une probabilité logarithmique en entrée et la normalise en la transformant en probabilité non logarithmique. Elle utilise la fonction d'exponentielle (math.exp) pour cela.<h3>
calculate_perplexiy() : </h3> Cette fonction calcule la perplexité d'un modèle de langue n-gramme sur un fichier de test. Elle prend en entrée le chemin du fichier de test, le modèle de langue n-gramme entraîné, la taille des n-grammes utilisée dans le modèle, et le vocabulaire utilisé pour remplacer les mots hors vocabulaire. La fonction lit chaque ligne du fichier de test, prétraite la phrase en la normalisant et en remplaçant les mots hors vocabulaire, puis calcule la probabilité logarithmique de chaque phrase selon le modèle de n-gramme. Enfin, elle utilise ces probabilités pour calculer la perplexité globale du modèle sur le fichier de test, qui est ensuite retourné.|

In [24]:
def normalize_log_prob(log_prob: float) -> float:
    """
    Normalise la probabilité logarithmique en soustrayant le logarithme de la somme des logarithmes des probabilités.

    Paramètres :
        log_prob (float) : La probabilité logarithmique à normaliser.

    Retourne :
        float : La probabilité normalisée.
    """
    return math.exp(log_prob)

def calculate_perplexity(test_file: str, ngram_model: dict, ngram_size: int, vocab: List[str]) -> float:
    """
    Calcule la perplexité d'un modèle de langue n-gramme sur un fichier de test.

    Paramètres :
        test_file (str) : Le chemin du fichier de test.
        ngram_model (dict) : Le modèle de langue n-gramme entraîné.
        ngram_size (int) : La taille des n-grammes utilisée dans le modèle.
        vocab (List[str]): Le vocabulaire utilisé pour remplacer les mots hors vocabulaire.

    Retourne :
        float : La perplexité calculée.
    """
    total_log_prob = 0.0
    total_words = 0

    with open(test_file, "r", encoding="utf-8") as file:
        for line in file:
            # Prétraitement de la phrase
            sentence = line.strip().lower()
            tokens = sentence.split()
            tokens = [token if token in vocab else "<unk>" for token in tokens]
            
            # Calcul de la probabilité logarithmique de la phrase
            for i in range(ngram_size - 1, len(tokens)):
                context = " ".join(tokens[i - ngram_size + 1:i])
                word = tokens[i]
                if context in ngram_model and word in ngram_model[context]:
                    total_log_prob += math.log(ngram_model[context][word])
                else:
                    pass
                
                total_words += 1
    
    # Calcul de la perplexité
    perplexity = math.exp(-total_log_prob / total_words)
    return perplexity

|<h1>Application</h1> |
| :------------------: |

In [25]:
if __name__ == "__main__":
    infile = "C:\\Users\\hp\\Downloads\\TP2_NLP\\PA_files\\ngramv1.train"
    test_file = "C:\\Users\\hp\\Downloads\\TP2_NLP\\PA_files\\ngramv1.test"
    test_sentence = "NOT IN A BOX"
    ngram_size = 2
    k = 0.01  # Valeur arbitraire pour le lissage add-k

    # Entraîner le modèle de n-gramme
    ngram_model = train(infile, ngram_size, k)

    # Utiliser la fonction predict_ngram pour prédire la probabilité logarithmique de la phrase de test
    log_sentence_prob = predict_ngram(test_sentence, ngram_model, ngram_size)
    print(f"Log Probability of the sentence '{test_sentence}': {log_sentence_prob:.4f}")

    # Normaliser la probabilité logarithmique
    sentence_prob = normalize_log_prob(log_sentence_prob)
    print(f"Probability of the sentence '{test_sentence}': {sentence_prob:.4f}")

    # Calculer la perplexité sur les données de test
    vocab = get_words_with_frequency_above_or_equal(process_data(open(infile).read())[1], 1)
    perplexity = calculate_perplexity(test_file, ngram_model, ngram_size, vocab)
    print(f"Perplexity on test data: {perplexity:.4f}")

Log Probability of the sentence 'NOT IN A BOX': -3.9223
Probability of the sentence 'NOT IN A BOX': 0.0198
Perplexity on test data: 1.8314
